In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
import tensorflow_datasets as tfds

In [3]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to C:\Users\benja\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

train_data, test_data = imdb["train"], imdb["test"]

In [8]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [11]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [12]:
# tokenize sentences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded  = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(), # can also try tf.keras.layers.GlobalAveragePooling1D() to flatten - bit faster, less accurate
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               245888    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 406,017
Trainable params: 406,017
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 10
# can see we are overfitting
history = model.fit(
    padded,
    training_labels_final,
    epochs=num_epochs,
    batch_size=32,
    validation_data=(testing_padded, testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4561 - accuracy: 0.7686 - val_loss: 0.3514 - val_accuracy: 0.8448
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1986 - accuracy: 0.9244 - val_loss: 0.4439 - val_accuracy: 0.8151
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0467 - accuracy: 0.9864 - val_loss: 0.6388 - val_accuracy: 0.8151
Epoch 4/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0079 - accuracy: 0.9985 - val_loss: 0.8101 - val_accuracy: 0.8082
Epoch 5/10
782/782 [==============================] - 3s 3ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.8712 - val_accuracy: 0.8176
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 2.6245e-04 - accuracy: 1.0000 - val_loss: 0.9172 - val_accuracy: 0.8188
Epoch 7/10
782/782 [==============================] - 3s 3ms/step - loss: 1.2765e-04 - accuracy: 1.0000 - val_loss: 0.9561 - val_accuracy: 0.819

In [19]:
e = model.layers[0]
weigths = e.get_weights()[0]
print(weigths.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [20]:
reverse_word_index = {value: key for key, value in word_index.items()}

In [21]:
import io

out_v = io.open("vecs.tsv", "w", encoding="utf-8")
out_m = io.open("meta.tsv", "w", encoding="utf-8")

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weigths[word_num]
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

# go to tensorboard word embedding projector and load the files created here to visualize